In [2]:
import tensorflow as tf
import sklearn as sk
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [3]:
df = pd.read_csv('/content/drive/MyDrive/Factor_Zc.csv', index_col=0)
df.head()

,Tat90_SGat0.55,Tat180_SGat0.55,Tat270_SGat0.55,Tat360_SGat0.55,Tat450_SGat0.55,Tat540_SGat0.55,Tat90_SGat0.6,Tat180_SGat0.6,Tat270_SGat0.6,Tat360_SGat0.6,...,Tat270_SGat0.85,Tat360_SGat0.85,Tat450_SGat0.85,Tat540_SGat0.85,Tat90_SGat0.9,Tat180_SGat0.9,Tat270_SGat0.9,Tat360_SGat0.9,Tat450_SGat0.9,Tat540_SGat0.9
Pressure(psia),,,,,,,,,,,,,,,,,,,,,
1,0.942285,0.995647,0.998879,0.999549,0.999774,0.999873,0.930689,0.994909,0.998704,0.999481,...,0.997503,0.999028,0.999519,0.999727,0.803353,0.988243,0.997186,0.998911,0.999463,0.999695
1783,0.752398,0.499859,0.406835,0.404793,0.642339,0.824271,0.773649,0.513877,0.415007,0.395755,...,0.462968,0.406388,0.406146,0.564014,0.898182,0.598839,0.473417,0.412684,0.401492,0.516956
1882,0.794117,0.527364,0.428642,0.422409,0.637798,0.820099,0.816552,0.542184,0.437382,0.414608,...,0.488288,0.427973,0.424554,0.564172,0.948018,0.631946,0.499347,0.434742,0.420765,0.521551
1981,0.835830,0.554845,0.450384,0.440038,0.635500,0.816735,0.859450,0.570468,0.459698,0.433373,...,0.513570,0.449481,0.442919,0.566837,0.997851,0.665039,0.525242,0.456728,0.439943,0.528381
2080,0.877536,0.582300,0.472062,0.457658,0.635202,0.814176,0.902342,0.598730,0.481957,0.452051,...,0.538816,0.470914,0.461232,0.571557,1.047681,0.698117,0.551103,0.478646,0.459028,0.536926


In [4]:
pressures = []
temperatures = []
specific_gravity = []
z_values = []

for col in df.columns:
    try:
        t_str, g_str = col.split('_')
        temp = float(t_str.split('Tat')[1])
        sg = float(g_str.split('SGat')[1])

        for p, z in zip(df.index, df[col]):
            pressures.append(p)
            temperatures.append(temp)
            specific_gravity.append(sg)
            z_values.append(z)
    except Exception as e:
        print(f'Error in columna: {col} - {e}')

df_long = pd.DataFrame({
    'Pressure': pressures,
    'Temperature': temperatures,
    'SG': specific_gravity,
    'Z': z_values
})

df_long = df_long.dropna()
df_long = df_long[df_long['Z'] > 0]

df_long.head()

,Pressure,Temperature,SG,Z
0,1,90.0,0.55,0.942285
1,1783,90.0,0.55,0.752398
2,1882,90.0,0.55,0.794117
3,1981,90.0,0.55,0.835830
4,2080,90.0,0.55,0.877536


In [5]:
df_long.tail(10)

,Pressure,Temperature,SG,Z
3062,7030,540.0,0.9,1.236155
3063,7129,540.0,0.9,1.250167
3064,7228,540.0,0.9,1.264157
3065,7327,540.0,0.9,1.278125
3066,7426,540.0,0.9,1.292072
3067,7525,540.0,0.9,1.305999
3068,7624,540.0,0.9,1.319904
3069,7723,540.0,0.9,1.333789
3070,7822,540.0,0.9,1.347653
3071,7921,540.0,0.9,1.361496


In [6]:
print(len(df_long))

3072


In [7]:
X = df_long[['Pressure', 'Temperature', 'SG']].values
y = df_long['Z'].values

In [8]:
scaler = sk.preprocessing.StandardScaler()
X_scaled = scaler.fit_transform(X)

In [9]:
X_train, X_test, y_train, y_test = sk.model_selection.train_test_split(X_scaled, y, test_size=0.2, random_state=42)

In [10]:
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(3,)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1)
])

In [11]:
model.compile(optimizer='adam', loss='mse')

In [12]:
print('Reloading model...')
history = model.fit(X_train, y_train, epochs=300, validation_split=0.1, verbose=False)
print('Finishing model...')

Reloading model...
Finishing model...


In [13]:
new_input = scaler.transform([[8000, 160, 0.75]])
z_pred = model.predict(new_input)
print(z_pred)
#print(f'Z predicted: {z_pred[0][0]}'

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 111ms/step
[[2.7344158]]


In [14]:
model.save('z_model_2025_04_15_v1.keras')

In [15]:
from google.colab import files
files.download('z_model_2025_04_15_v1.keras')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [16]:
import joblib

joblib.dump(scaler, "scaler_z_2025_04_15.pkl")

['scaler_z_2025_04_15.pkl']

In [17]:
from google.colab import files
files.download("scaler_z_2025_04_15.pkl")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>